In [2]:
from scripts.StreamProcessing import Stream_Data # found in the scripts folder inside StreamProcessing.py. This class uses PySpark

PySpark found


In [ ]:
# Initialize the Stream_Data Class using the topic name established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
temperature_config = Stream_Data(topic='temperature', host='localhost:9092')
temperatures = temperature_config.subsribe()
temperatures

23/05/07 11:06:06 WARN Utils: Your hostname, camagakhan-Inspiron-7577 resolves to a loopback address: 127.0.1.1; using 192.168.6.24 instead (on interface wlp60s0)
23/05/07 11:06:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 11:06:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
